In [27]:
import pandas as pd

In [5]:
!unzip ./data/covid-19_zip\ \(1\).zip

Archive:  ./data/covid-19_zip (1).zip
  inflating: archive/countries-aggregated.csv  
  inflating: archive/key-countries-pivoted.csv  
  inflating: archive/reference.csv   
  inflating: archive/time-series-19-covid-combined.csv  
  inflating: archive/us_confirmed.csv  
  inflating: archive/us_deaths.csv   
  inflating: archive/us_simplified.csv  
  inflating: archive/worldwide-aggregate.csv  
  inflating: data/validation_report.json  
  inflating: data/countries-aggregated_csv.csv  
  inflating: data/key-countries-pivoted_csv.csv  
  inflating: data/reference_csv.csv  
  inflating: data/time-series-19-covid-combined_csv.csv  
  inflating: data/us_confirmed_csv.csv  
  inflating: data/us_deaths_csv.csv  
  inflating: data/us_simplified_csv.csv  
  inflating: data/worldwide-aggregate_csv.csv  
  inflating: data/countries-aggregated_json.json  
  inflating: data/key-countries-pivoted_json.json  
  inflating: data/reference_json.json  
  inflating: data/time-series-19-covid-combined_json.j

In [28]:
!ls

data  Untitled.ipynb


In [29]:
!ls ./data/

countries-aggregated.csv  time-series-19-covid-combined.csv


In [30]:
test_1 = pd.read_csv('./data/countries-aggregated.csv')

In [31]:
test_2 = pd.read_csv('./data/time-series-19-covid-combined.csv')

In [32]:
data = test_2

In [33]:
data.head()

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,0,0.0,0
1,2020-01-23,Afghanistan,NaN,0,0.0,0
2,2020-01-24,Afghanistan,NaN,0,0.0,0
3,2020-01-25,Afghanistan,NaN,0,0.0,0
4,2020-01-26,Afghanistan,NaN,0,0.0,0


In [34]:
data.drop('Province/State',axis=1,inplace=True)

In [35]:
data.isna().sum()

Date                 0
Country/Region       0
Confirmed            0
Recovered         6096
Deaths               0
dtype: int64

In [43]:
import numpy as np
data['Recovered'] = data['Recovered'].fillna(np.mean(data['Recovered']))

In [44]:
data.isna().sum()

Date              0
Country/Region    0
Confirmed         0
Recovered         0
Deaths            0
dtype: int64

In [47]:
data.dtypes

Date               object
Country/Region     object
Confirmed           int64
Recovered         float64
Deaths              int64
dtype: object

In [48]:
recoverd = []
for recover in data['Recovered']:
    recoverd.append(int(recover))

In [49]:
data['Recovered'] = recoverd

In [50]:
data.dtypes

Date              object
Country/Region    object
Confirmed          int64
Recovered          int64
Deaths             int64
dtype: object

In [51]:
dates = []
for date in data['Date']:
    dates.append(int(date.replace('-','')))

In [53]:
data['Date'] = dates

In [54]:
data.head()

,Date,Country/Region,Confirmed,Recovered,Deaths
0,20200122,Afghanistan,0,0,0
1,20200123,Afghanistan,0,0,0
2,20200124,Afghanistan,0,0,0
3,20200125,Afghanistan,0,0,0
4,20200126,Afghanistan,0,0,0


In [55]:
def object_to_int(data,coloum):
    index = -1
    info_dict = {}
    all_info = []
    for i in data[coloum]:
        if i not in info_dict:
            index = index + 1
            info_dict[i] = index
    for x in data[coloum]:
        all_info.append(info_dict[x])
    return all_info,info_dict

In [56]:
info = object_to_int(data,'Country/Region')
with open('city.json','w') as json_file:
    import json
    json.dump(info[1],json_file)
data['Country/Region'] = info[0]

In [57]:
data.head()

,Date,Country/Region,Confirmed,Recovered,Deaths
0,20200122,0,0,0,0
1,20200123,0,0,0,0
2,20200124,0,0,0,0
3,20200125,0,0,0,0
4,20200126,0,0,0,0


In [59]:
data.dtypes

Date              int64
Country/Region    int64
Confirmed         int64
Recovered         int64
Deaths            int64
dtype: object

In [122]:
X = data.drop(['Confirmed','Deaths','Recovered'],axis=1)
y = data.drop(['Date','Country/Region'],axis=1)

In [123]:
from sklearn.model_selection import *
from sklearn.metrics import *

In [124]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

ValueError: Found input variables with inconsistent numbers of samples: [208026, 312039]

In [89]:
from sklearn.tree import *
from sklearn.ensemble import *

In [90]:
model_RandomForestRegressor = RandomForestRegressor().fit(X_train,y_train)
model_RandomForestRegressor.score(X_test,y_test)

0.9712875000562974

In [91]:
model_DecisionTreeRegressor = DecisionTreeRegressor().fit(X_train,y_train)
model_DecisionTreeRegressor.score(X_test,y_test)

0.9706971252349169

In [92]:
info[1]['Sri Lanka']

163

In [130]:
array = np.array([[20210207,163]])
# array.reshape(-1,1)

In [131]:
model_RandomForestRegressor.predict(array)

array([[67135.95333333, 60617.338     ,   338.55133333]])

In [133]:
import pickle

In [134]:
pickle.dump(model_DecisionTreeRegressor,open('./model_DTR.pkl','wb'))
pickle.dump(model_RandomForestRegressor,open('./model_RFR.pkl','wb'))

In [143]:
grid = {
    "max_depth":[None,5,10,15,20,25,30,35,40,45,50],
    'splitter':['best','random'],
    "max_features":["auto","sqrt",'log2'],
    "min_samples_leaf":[2,4,6,8,10],
}
model_rs = DecisionTreeRegressor()
rs = RandomizedSearchCV(estimator=model_rs,param_distributions=grid,cv=5,verbose=25,n_iter=25)
rs.fit(X_train,y_train)
print(rs.best_estimator_)
print(rs.score(X_test,y_test))
print(rs.best_score_)
pickle.dump(rs,open('./model_DTR_random.pkl','wb'))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None 
[CV]  splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None, score=0.906, total=   0.0s
[CV] splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None 
[CV]  splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None, score=0.928, total=   0.0s
[CV] splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None 
[CV]  splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None, score=0.944, total=   0.0s
[CV] splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None 
[CV]  splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None, score=0.934, total=   0.0s
[CV] splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


[CV]  splitter=random, min_samples_leaf=4, max_features=auto, max_depth=None, score=0.955, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5 
[CV]  splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5, score=0.050, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5 
[CV]  splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5, score=0.069, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5 
[CV]  splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5, score=0.044, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5 
[CV]  splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5, score=0.036, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5 
[CV]  splitter=random, min_samples_leaf=6, max_features=auto, max_depth=5, score=0.206, total=   0.0s
[CV] splitter=b

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.4s remaining:    0.0s


[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=45, score=0.867, total=   0.1s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=45 
[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=45, score=0.764, total=   0.1s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=45 
[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=45, score=0.670, total=   0.1s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=45 


...

[CV]  splitter=random, min_samples_leaf=6, max_features=auto, max_depth=None, score=0.938, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=auto, max_depth=None 
[CV]  splitter=random, min_samples_leaf=6, max_features=auto, max_depth=None, score=0.936, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20 
[CV]  splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20, score=0.256, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20 
[CV]  splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20, score=0.398, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20 
[CV]  splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20, score=0.312, total=   0.0s
[CV] splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20 
[CV]  splitter=random, min_samples_leaf=6, max_features=log2, max_depth=20, score=0.473, total=   0.0s
[

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    1.1s remaining:    0.0s


[CV]  splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50, score=0.971, total=   0.1s
[CV] splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50 
[CV]  splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50, score=0.976, total=   0.1s
[CV] splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50 
[CV]  splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50, score=0.976, total=   0.1s
[CV] splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50 
[CV]  splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50, score=0.977, total=   0.1s
[CV] splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50 
[CV]  splitter=best, min_samples_leaf=6, max_features=auto, max_depth=50, score=0.977, total=   0.1s
[CV] splitter=random, min_samples_leaf=4, max_features=log2, max_depth=45 
[CV]  splitter=random, min_samples_leaf=4, max_features=log2, max_depth=45, score=0.322, total=   0.0s
[CV] splitter=random, min

[CV]  splitter=random, min_samples_leaf=10, max_features=sqrt, max_depth=50, score=0.388, total=   0.0s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50 
[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50, score=0.906, total=   0.1s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50 
[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50, score=0.775, total=   0.1s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50 
[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50, score=0.934, total=   0.1s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50 
[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50, score=0.665, total=   0.1s
[CV] splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50 
[CV]  splitter=best, min_samples_leaf=2, max_features=sqrt, max_depth=50, score=0.791, total=   0.1s
[CV] splitter=best, min_sa

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    6.8s finished


In [142]:
date = input('Date Like 20210207 #Year#Month#Day \n ')
country = input('Country Web version comming soon \n ')
array = np.array([[date,country]])
print(rs.predict(array))
print(model_RandomForestRegressor.predict(array))
print(model_DecisionTreeRegressor.predict(array))

Date Like 20210207 #Year#Month#Day 
 20210208
Country 163 is Sri Lanka Web version comming soon 
 163
[[67482.5 61014.    341. ]]
[[67135.95333333 60617.338        338.55133333]]
[[67482.5 61014.    341. ]]


# the predictions may change according to different reasons. the predictions may change the 97% accuracy this shows may change the predictions may change